In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from tifffile import imread, imwrite
import numpy as np
import pandas as pd
from skimage.draw import polygon
from pathlib import Path
from loguru import logger
from IPython.display import clear_output
import spatialdata as sd
import napari
from magicgui import magicgui
from napari_spatialdata import Interactive

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.core_cutting.channel_scanner import discover_channels, build_transfer_map
from multiplex_pipeline.core_cutting.controller import CorePreparationController
from multiplex_pipeline.core_cutting.file_io import GlobusFileStrategy
from multiplex_pipeline.utils.globus_utils import GlobusConfig, create_globus_tc
from multiplex_pipeline.utils.file_utils import GlobusPathConverter

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
# define pathways
settings_path = r'C:\BLCA-7_Analysis\analysis_settings_BLCA7.yaml'

### Load analysis settings

In [3]:
# load analysis configuration
settings = load_analysis_settings(settings_path)
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/BLCA-7/BLCA-7_Final', analysis_name='BLCA-7_Analysis', local_analysis_dir='C:/', remote_analysis_dir='/ix1/kkedziora/blca_analysis', log_dir=None), core_detection=CoreDetectionSettings(detection_image='BLCA-7_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=['008_ECad'], use_markers=['DAPI', 'CD45', 'CD44', 'HLA1', 'NaKATPase', 'CD11C', 'pCK26', 'pS6', 'HES1'], ignore_markers=['Antibody1', 'TNFa', 'Snail1', 'SKP2', 'ProgRc', 'Plk1', 'PH3', 'PDL1', 'p65', 'p130', 'p-p130', 'p-Cdc6', 'LAG3', 'IL-8', 'HER2', 'ERa', 'EpCAM', 'E2F1', 'cycD3', 'cycB2', 'CDC25C', 'CD86', 'CD73', 'CD69', 'CD62L', 'CD56', 'CD4', 'CD25', 'CD19', 'CD27', 'CCR7', 'cCASP3'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanu

In [4]:
core_dir = settings.analysis_dir / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/BLCA-7_Analysis/cores/Core_000.zarr')]

In [162]:
for sd_path in path_list[:1]:

    # get sdata
    sdata = sd.read_zarr(sd_path)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


In [165]:
%load_ext autoreload
%autoreload 2
from multiplex_pipeline.widgets.qc_widget import QCWidget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [166]:
viewer = napari.Viewer()

qc_widget = QCWidget(viewer,sdata)

viewer.window.add_dock_widget(qc_widget, area='right')

In [161]:
qc_widget.viewer.status = f'{self.shapes_name} has been saved to disk.'

In [169]:
sdata

SpatialData object, with associated Zarr store: C:\BLCA-7_Analysis\cores\Core_000.zarr
├── Images
│     ├── 'CD11C': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
│     ├── 'CD44': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
│     ├── 'CD45': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
│     ├── 'DAPI': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
│     ├── 'HES1': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
│     ├── 'HLA1': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
│     ├── 'NaKATPase': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
│     ├── 'pCK26': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
│     └── 'pS6': DataTree[cyx] (1, 4992, 5312), (1, 2496, 2656), (1, 1248, 1328)
└── Labels
      ├── 'blob': DataTree[yx] (4992, 5312), (2496, 2656), (1248, 1328)
      ├── 'cytoplasm': DataTree[yx] (4992, 5312), (2496, 2656), (1248, 1328

In [148]:
qc_widget.shapes_name

'qc_exclude_CD44'

In [128]:
qc_widget.position

4

In [114]:
data = [np.array(self.sdata[self.shapes_name].geometry[i].exterior.coords) for i in range(len(self.sdata[self.shapes_name]))]
data

[array([[ 979.4635029 , 1296.84862944],
        [ 979.4635029 , 2350.36559995],
        [1865.95949027, 2350.36559995],
        [1865.95949027, 1296.84862944],
        [ 979.4635029 , 1296.84862944]]),
 array([[2341.32690379, 2915.66738899],
        [2341.32690379, 4014.15154726],
        [3420.53941016, 4014.15154726],
        [3420.53941016, 2915.66738899],
        [2341.32690379, 2915.66738899]])]

In [118]:
data[0].shape

(5, 2)

In [119]:
viewer.add_shapes(data = data, name = self.shapes_name, shape_type = 'polygon')

<Shapes layer 'qc_exclude_CD44' at 0x25f26d73f90>

In [72]:
import numpy as np
from geopandas import GeoDataFrame
from shapely import Polygon
from spatialdata.models import ShapesModel
from spatialdata.transformations import Identity

In [73]:
self = qc_widget        
if self.shapes_name in [x.name for x in self.viewer.layers]:
    print('ues')
            
    gdf = GeoDataFrame({"geometry": [self.numpy_to_shapely(x) for x in self.viewer.layers[self.shapes_name].data]})
    gdf = ShapesModel.parse(gdf, transformations={"aligned": Identity()})
    self.sdata.shapes[self.shapes_name] = gdf

ues


TypeError: numpy_to_shapely() takes 1 positional argument but 2 were given

In [75]:
self.viewer.layers[self.shapes_name].data

[array([[1993.74091548, 1798.59965193],
        [1993.74091548, 3395.10615174],
        [3420.82223571, 3395.10615174],
        [3420.82223571, 1798.59965193]])]

In [76]:
[self.numpy_to_shapely(x) for x in self.viewer.layers[self.shapes_name].data]

TypeError: numpy_to_shapely() takes 1 positional argument but 2 were given

In [67]:
[x.name for x in self.viewer.layers]

['CD44', 'qc_exclude_CD44']